# Skyline Optimization Project

### Importing Libraries

In [1]:
import gurobipy as gb
from gurobipy import *
import pandas as pd
import numpy as np
import os
import re

### Importing Data

In [ ]:
beverage_sales = pd.read_excel("https://raw.githubusercontent.com/lucas-penney/MGSC-662-Skyline-Optimization/main/Beverage%20Sales%20-%20Nov%202024%20-%20Oct%202025.xlsx", sheet_name=None)
wine_costing = pd.read_excel("https://raw.githubusercontent.com/lucas-penney/MGSC-662-Skyline-Optimization/main/Beverage%20Costing.xlsx")



In [3]:
wine_costing.head()

,Item,Glass ($),COGS,Bottle,COGS.1,COST,LEAD TIME\n(DAYS)
0,WINE,NaN,NaN,NaN,NaN,NaN,NaN
1,Champagne (G - 120ml),NaN,NaN,NaN,NaN,NaN,NaN
2,"Veuve Clicquot Rosé, Reims, France",NaN,NaN,198.0,0.617573,106.33,3.0
3,"Veuve Clicquot, Reims, France",NaN,NaN,156.0,0.475186,64.46,3.0
4,"Moët Chandon, Épernay, France",23.0,0.44968,124.0,0.521302,56.21,3.0


In [4]:
#Viewing what the first dataframe looks like
beverage_sales['Dec-24'].head()

,LOCATION_ID,LOCATION,PRODUCT_ID,CATEGORY,ATTRIBUTE,PRODUCT,DISPLAY_ORDER,DISPLAY_ORDER_1,DISPLAY_ORDER_2,QUANTITY,QTY_REDEEMED,QTY_PREPRINTED,NETT_AMOUNT,NETT_TAX_AMOUNT
0,8,Buffet Rest/Bar,22918,BEVA - Wine | White,Sauvignon Blanc,Cloudy Bay SB 750ml,5,1,14,4,0,0,336.0,43.84
1,8,Buffet Rest/Bar,26267,BEVA - Wine | White,Sauvignon Blanc,Stonel Rapaura SB 750ml,5,1,15,2,0,0,148.0,19.30
2,8,Buffet Rest/Bar,22916,BEVA - Wine | White,Sauvignon Blanc,Triplebank SB 750ml,5,1,17,6,0,0,360.0,46.98
3,8,Buffet Rest/Bar,30900,BEVA - Wine | White,Sauvignon Blanc,Ant Moore Sauvignon Blanc 750ml,5,1,18,1,0,0,58.0,7.57
4,8,Buffet Rest/Bar,14663,BEVA - Wine | White,Sauvignon Blanc,Rabbit Ranch SB 750ml,5,1,20,11,0,0,638.0,83.27


## Combining the Monthly Sales Data into a Single Dataframe
- The data was provided as an Excel file, where each tab was a different month and included sales of all beverages, beyond wine. Within each month, a wine could appear more than once if it was sold in more than one location (eg. cafe, restaurant, market kitchen)
- First, we filtered to only include wine, cleaned up the product names and created a long format dataframe with all the wines sold, quantities and their respective months
- Then, we pivoted the monthly data to wide format, creating a dataframe with the months as column, wines as rows and the entries being the sales
- Columns were added for sales over the 11 month period we had data for, as well as being extrapolated linearly to 12 months of sales

In [5]:
# Placeholder for the column that identifies the 75 types of wine
WINE_IDENTIFIER_COLUMN = "PRODUCT"

all_months_data = []
target_category = 'BEVA - Wine'

#Creates a long format dataframe that combines all the monthly sales data, creating a 'Product' 'Category', 'Quantity' and 'Month' column
for month, df in beverage_sales.items():
    # 1. Filter: Keep only rows where CATEGORY contains 'BEVA - WINE'
    df_filtered = df[
        df['CATEGORY'].str.contains(
            target_category, 
            case=False,  # Set to False to ignore case (e.g., 'Beva - Wine' will match 'BEVA - WINE')
            na=False     # Set to False to treat NaN values as non-matches (prevents errors)
        )
    ].copy()
    
    #Remove asterisks from the product column, whitespace
    df_filtered['PRODUCT'] = df_filtered['PRODUCT'].astype(str).str.replace('*', '', regex=False)
    df_filtered['PRODUCT'] = df_filtered['PRODUCT'].str.strip()

    # 2. Select: Keep only the essential columns (Wine ID, Category, Quantity)
    # Keep 'CATEGORY' for validation, and the identifier and quantity for the final output.
    df_selected = df_filtered[[WINE_IDENTIFIER_COLUMN, 'CATEGORY', 'QUANTITY']]
    

    df_aggregated = df_selected.groupby(WINE_IDENTIFIER_COLUMN, as_index=False)['QUANTITY'].sum()
    
    # 3. Add the 'Month' column
    df_aggregated['Month'] = month
    
    # 4. Add the processed DataFrame to the list
    all_months_data.append(df_aggregated)

# Concatenate all monthly DataFrames into one large DataFrame
combined_df = pd.concat(all_months_data, ignore_index=True)

#Display the number of unique wines they sold across all months
total_unique_wines = combined_df[WINE_IDENTIFIER_COLUMN].nunique()
print(f"Total unique wines across all months: {total_unique_wines}\n")

# Print a preview of the combined, filtered data
print("--- Preview of Combined, Filtered Data ---")
print(combined_df.head())
print(f"\nTotal rows in long format: {len(combined_df)}\n")

# Pivot the Data  from long format to wide format

final_df = combined_df.pivot_table(
    index=WINE_IDENTIFIER_COLUMN,  # The 75 wine types become the rows
    columns='Month',               # The months become the columns
    values='QUANTITY',             # The sales figures are the cell values
    aggfunc='sum'                  # This sums duplicates (e.g., 2 sales of 'Cabernet' in Jan)
).reset_index()



#Dropping the 'Year' column, since it doesnt acccurately reflect sales over the period
final_df = final_df.drop(columns=['Year'])

# Handle NaNs (if some wines weren't sold in a month)
# By default, pivot_table will leave NaN. You can replace them with 0.
final_df = final_df.fillna(0)

#Fixes the order of the columns
month_order = [
    'Dec-24', 'Jan-25', 'Feb-25', 'Mar-25', 'Apr-25', 'May-25',
    'Jun-25', 'Jul-25', 'Aug-25', 'Sep-25', 'Oct-25'
]

# 2. Re-order the DataFrame columns
# This keeps the PRODUCT column and adds the months in the correct order
final_df = final_df[[WINE_IDENTIFIER_COLUMN] + month_order]

#Removes name of index column
final_df.columns.name = None

#Identifies all columns that arent wine names as those to be summed
month_columns = [col for col in final_df.columns if col != WINE_IDENTIFIER_COLUMN]

# Creates a new 'TotalSales_11Months' column, since we have 11 months of data
final_df['TotalSales_11Months'] = final_df[month_columns].sum(axis=1)

# Define a scaling factor to convert 11 months of data to annual
scaling_factor = 12 / 11

# Create 'AdjustedAnnualSales' column, round to whole number
final_df['AdjustedAnnualSales'] = final_df['TotalSales_11Months'] * scaling_factor
final_df['AdjustedAnnualSales'] = final_df['AdjustedAnnualSales'].round(0)

# Display the final, clean DataFrame
print("--- Combined and Filtered Wide Format Dataframe ---")
print(final_df.head())
print(f"\nTotal rows in wide format: {len(final_df)}\n")

Total unique wines across all months: 148

--- Preview of Combined, Filtered Data ---
                           PRODUCT  QUANTITY   Month
0        Anniversary Bubbles 150ml        61  Dec-24
1           Ant Moore Chardy 150ml        65  Dec-24
2           Ant Moore Chardy 750ml         2  Dec-24
3               Ant Moore SB 150ml        33  Dec-24
4  Ant Moore Sauvignon Blanc 750ml         1  Dec-24

Total rows in long format: 1120

--- Combined and Filtered Wide Format Dataframe ---
                           PRODUCT  Dec-24  Jan-25  Feb-25  Mar-25  Apr-25  \
0        Anniversary Bubbles 150ml    61.0    88.0    85.0    85.0    58.0   
1           Ant Moore Chardy 150ml    65.0    87.0    86.0   102.0    56.0   
2           Ant Moore Chardy 750ml     2.0     1.0     4.0     3.0     1.0   
3               Ant Moore SB 150ml    33.0    69.0    59.0    84.0    54.0   
4  Ant Moore Sauvignon Blanc 750ml     1.0     6.0     2.0     9.0     4.0   

   May-25  Jun-25  Jul-25  Aug-25  Sep-25

In [6]:
# To confirm the data was correctly translated from the Excel file 
# Checks that the Rabbit Ranch wine sales is equal to 40
# The Rabbit Ranch SB 750ml wine was sold in both the cafe and the restaurant in October 2025, so this confirms these two entries were combined appropriately and are in the right row/column
product_row = final_df.query("PRODUCT == 'Rabbit Ranch SB 750ml'")
print(product_row["Oct-25"].iloc[0])

40.0


## Merging Lead Times into Sales Dataframe

In [7]:
def normalize_wine_name(name):
    """A helper function to clean wine names."""
    if not isinstance(name, str):
        return ""
    
    #Removes text after the comma, which is region names in the dataset
    name = name.split(',')[0]

    #convert to lower case
    name = name.lower()  

    #remove leading and trailing whitespace
    name = name.strip()
    
    # Remove punctuation
    name = name.replace('.', '').replace(',', '').replace("'", "")
            
    # Remove extra spaces
    name = ' '.join(name.split())

    # Remove suffix volumes (with or without spaces/trailing numbers)
    # e.g., "Ant Moore Chardy 150ml" -> "Ant Moore Chardy"
    # e.g., "Brancott Brut 750ml9" -> "Brancott Brut"
    name = re.sub(r'\s*\b\d+ml\d*\b', '', name)
    
    return name

#### Clean Wine Names in Sales Dataframe

In [8]:
final_df['Product_Clean'] = final_df['PRODUCT'].apply(normalize_wine_name)
final_df['Product_Clean'][0:11]

0           anniversary bubbles
1              ant moore chardy
2              ant moore chardy
3                  ant moore sb
4     ant moore sauvignon blanc
5             b late harvest sb
6             bound kings rd pn
7          boundary kings rd pn
8                 brancott brut
9                brancott cuvee
10       brancott estate merlot
Name: Product_Clean, dtype: object

#### Clean Wine Names in Costing/Lead Times Dataframe

In [9]:
#Removing whitespace from the column names
wine_costing.columns = wine_costing.columns.str.strip()

In [10]:
#Standardize wine names in the costing dataframe
wine_costing['wine_name_clean'] = wine_costing['Item'].apply(normalize_wine_name)

wine_costing['wine_name_clean'].head()

0                   wine
1        champagne (g -)
2    veuve clicquot rosé
3         veuve clicquot
4           moët chandon
Name: wine_name_clean, dtype: object

In [11]:
# Adding wine category to the end of the wine name to match other df, creating a mapping system to match up closest to other df
#Created a new column called 'wine_name_full' which should be close to naming in other df

# 1. Identify Category Rows
wine_costing['Wine_Category'] = wine_costing.apply(
    # Use the new lowercase 'wine_name_clean' column
    lambda row: row['wine_name_clean'] if pd.isna(row['Bottle']) else pd.NA,
    axis=1
)

# 2. Forward Fill (ffill)
wine_costing['Wine_Category'] = wine_costing['Wine_Category'].ffill()

# 3. --- FIX: Define and Apply Custom Mapping with LOWERCASE KEYS ---
# The keys must be lowercase to match the data in the 'Wine_Category' column.
category_map = {
    'rose': ' rose',
    'sauvignon blanc': ' sb',
    'pinot gris': ' pg',
    'riesling': ' riesling',
    'chardonnay': ' chardy',
    'pinot noir': ' pn',
    'cabernet sauvignon': ' cab sauv',
    'shiraz/syrah': ' shiraz',
    'merlot': ' merlot',
    'merlot blends': ' merlot',
    
    # Categories to ignore (lowercase keys)
    'champagne (g - 120ml)': None,
    'méthode traditionelle (g -120ml)': None,
    'other wine': None,
    'dessert wine': None,
    'wine': None, 
    'white wine': None,
    'red wine': None
}

# Map the 'Wine_Category' (which is lowercase) to the desired suffix.
wine_costing['Category_Suffix'] = wine_costing['Wine_Category'].map(category_map).fillna('')

# 4. Filter for actual wine items
wine_items = wine_costing['Bottle'].notna()

# 5. --- MODIFIED STEP: Concatenate with the new Suffix ---
# We use the 'wine_name_clean' column and create 'wine_name_full'
wine_costing.loc[wine_items, 'wine_name_full'] = (
    wine_costing['wine_name_clean'] + wine_costing['Category_Suffix']
)

# Handle non-wine items (fill 'wine_name_full' with original 'wine_name_clean' name)
wine_costing['wine_name_full'] = wine_costing['wine_name_full'].fillna(wine_costing['wine_name_clean'])

# 6. Display the result
print("Sample of Categorized Items (Corrected):")
output_cols = ['wine_name_clean', 'Bottle', 'Wine_Category', 'Category_Suffix', 'wine_name_full']
print(wine_costing[output_cols].head(20))

Sample of Categorized Items (Corrected):
                       wine_name_clean  Bottle                Wine_Category  \
0                                 wine     NaN                         wine   
1                      champagne (g -)     NaN              champagne (g -)   
2                  veuve clicquot rosé   198.0              champagne (g -)   
3                       veuve clicquot   156.0              champagne (g -)   
4                         moët chandon   124.0              champagne (g -)   
5          méthode traditionelle (g -)     NaN  méthode traditionelle (g -)   
6                         pelorus brut   101.0  méthode traditionelle (g -)   
7               jacob’s creek prosecco    75.0  méthode traditionelle (g -)   
8           brancott estate brut cuvee    60.0  méthode traditionelle (g -)   
9                           other wine     NaN                   other wine   
10          domaine thomson “ez chêne”   220.0                   other wine   
11         

In [12]:
#Drop the headers from the dataframe

#Selects all of the headers from the dataframe, which are the categories of wine
values_to_drop = wine_costing['Wine_Category'].unique()

# 2. Use .isin() to find rows matching the list
# The ~ (tilde) inverts the selection, keeping only the rows that are NOT in the list
wine_costing = wine_costing[~wine_costing['wine_name_full'].isin(values_to_drop)]

# You can now check the DataFrame
print(wine_costing.tail(10))

                                           Item  Glass ($)      COGS  Bottle  \
74                     Stoneleigh , Marlborough       18.0  0.175056    78.0   
75                 Brancott Estate, Hawke’s Bay       15.0  0.169587    63.0   
77       Church Road Grand Reserve, Hawke’s Bay        NaN       NaN   102.0   
78                   Church Road, Hawke’s Bay\t       18.0  0.183361    72.0   
79                  Festival Block, Hawke’s Bay       15.0  0.134933    63.0   
81                          St Hugo, Coonawarra        NaN       NaN   120.0   
82  Jacob’s Creek Double Barrel, Barossa Valley       16.0  0.232156    72.0   
84       Church Road Grand Reserve, Hawke’s Bay        NaN       NaN   108.0   
85               The Last Shepherd, Hawke’s Bay        NaN       NaN    84.0   
86        Jacob’s Creek Reserve, Barossa Valley       17.0  0.165329    77.0   

      COGS.1   COST  LEAD TIME\n(DAYS)              wine_name_clean  \
74  0.201987  13.70                1.0          

#### Creating a Matching Function to Match Wine Names in Sales and Costing/Lead Time Dataframes

In [13]:
from thefuzz import process
from thefuzz import fuzz

# Define the matching function (no change needed) ---
def find_best_match(Product_Clean, costs_master_list):
    """
    Finds the best match for a single wine name from the master costs list.
    """
    # extractOne returns a tuple: (best_match_string, similarity_score)
    best_match, score = process.extractOne(Product_Clean, costs_master_list, scorer=fuzz.token_set_ratio)
    
    # We set a threshold to avoid bad matches. Adjust 85 as needed.
    if score >= 85:
        return best_match
    else:
        # Return None if no good match is found
        return None

# --- B. Create the master list from your costs file (no change needed) ---
costs_wine_list = wine_costing[wine_costing['Bottle'].notna()]['wine_name_full'].unique()

# --- C. Apply the function to your sales data (Refactored) ---
# This creates a new 'cost_match' column in your 'final_df'.
# Assumes 'final_df' has a 'sales_wine_name' column.
final_df['cost_match'] = final_df['Product_Clean'].apply(
    lambda x: find_best_match(x, costs_wine_list)
)

#You can check the intermediate matches
print(final_df[['Product_Clean', 'cost_match']].head(10))




               Product_Clean                  cost_match
0        anniversary bubbles                        None
1           ant moore chardy            ant moore chardy
2           ant moore chardy            ant moore chardy
3               ant moore sb                ant moore sb
4  ant moore sauvignon blanc                ant moore sb
5          b late harvest sb                        None
6          bound kings rd pn                        None
7       boundary kings rd pn                        None
8              brancott brut  brancott estate brut cuvee
9             brancott cuvee  brancott estate brut cuvee


In [14]:
final_df.head()

,PRODUCT,Dec-24,Jan-25,Feb-25,Mar-25,Apr-25,May-25,Jun-25,Jul-25,Aug-25,Sep-25,Oct-25,TotalSales_11Months,AdjustedAnnualSales,Product_Clean,cost_match
0,Anniversary Bubbles 150ml,61.0,88.0,85.0,85.0,58.0,42.0,54.0,59.0,49.0,57.0,78.0,716.0,781.0,anniversary bubbles,None
1,Ant Moore Chardy 150ml,65.0,87.0,86.0,102.0,56.0,36.0,34.0,75.0,42.0,47.0,73.0,703.0,767.0,ant moore chardy,ant moore chardy
2,Ant Moore Chardy 750ml,2.0,1.0,4.0,3.0,1.0,2.0,1.0,2.0,1.0,3.0,0.0,20.0,22.0,ant moore chardy,ant moore chardy
3,Ant Moore SB 150ml,33.0,69.0,59.0,84.0,54.0,20.0,23.0,49.0,49.0,33.0,44.0,517.0,564.0,ant moore sb,ant moore sb
4,Ant Moore Sauvignon Blanc 750ml,1.0,6.0,2.0,9.0,4.0,1.0,2.0,2.0,2.0,0.0,0.0,29.0,32.0,ant moore sauvignon blanc,ant moore sb


#### Merge Lead Times from Costing/Lead Times Dataframe to the Sales Dataframe

In [15]:
# 1. Select only the necessary columns from your costs DataFrame
# We need the merge key ('wine_name_full') and the value you want ('LEAD TIME\n(DAYS)')
df_costs_to_merge = wine_costing[['wine_name_full', 'LEAD TIME\n(DAYS)']]

# 2. Perform the left merge
# This keeps all rows from final_df and adds the lead time where a match was found.
final_df = pd.merge(
    final_df,                 # Left DataFrame (your sales data)
    df_costs_to_merge,        # Right DataFrame (just the columns you need)
    left_on='cost_match',     # Key from final_df
    right_on='wine_name_full',# Key from wine_costing
    how='left'                # Keep all sales records
)

# 3. Clean up (optional)
# The merge creates a 'wine_name_full' column. We can drop it as it's redundant.
if 'wine_name_full' in final_df.columns:
    final_df = final_df.drop(columns='wine_name_full')

# 4. Display the result to confirm
print(final_df.head(10))

                           PRODUCT  Dec-24  Jan-25  Feb-25  Mar-25  Apr-25  \
0        Anniversary Bubbles 150ml    61.0    88.0    85.0    85.0    58.0   
1           Ant Moore Chardy 150ml    65.0    87.0    86.0   102.0    56.0   
2           Ant Moore Chardy 750ml     2.0     1.0     4.0     3.0     1.0   
3               Ant Moore SB 150ml    33.0    69.0    59.0    84.0    54.0   
4  Ant Moore Sauvignon Blanc 750ml     1.0     6.0     2.0     9.0     4.0   
5           B Late Harvest SB 60ml     0.0     0.0     1.0     3.0     0.0   
6         Bound. Kings Rd PN 750ml     3.0     0.0     6.0     1.0     2.0   
7       Boundary Kings Rd PN 150ml   104.0   137.0   131.0   113.0    89.0   
8              Brancott Brut 750ml     1.0     0.0    20.0     0.0     1.0   
9             Brancott Cuvee 200ml    64.0    76.0    50.0     4.0     0.0   

   May-25  Jun-25  Jul-25  Aug-25  Sep-25  Oct-25  TotalSales_11Months  \
0    42.0    54.0    59.0    49.0    57.0    78.0                71

<!-- # --- 4. 🔗 Merge Your Data (Refactored) ---

# Select the columns you want from your costs file
df_costs_to_merge = wine_costing[
    ['wine_name_clean', 'COGS', 'LEAD TIME\n(DAYS)'] # Add any other cost columns
]

# Merge 'final_df' with the costs data
# The final merged DataFrame is assigned back to 'final_df'
final_df = pd.merge(
    final_df,                  # Use 'final_df' as the left DataFrame
    df_costs_to_merge,
    left_on='cost_match',      # From 'final_df'
    right_on='wine_name_clean', # From df_costs_to_merge
    how='left'                 # Keep all sales, even if no cost match
)

# You'll have your sales data enriched with cost info
print(final_df.head()) -->

In [19]:
#Dropping Product_Clean and cost_match columns
final_df = final_df.drop(columns=['Product_Clean', 'cost_match'], errors='ignore')

#Rename Lead Time column
final_df = final_df.rename(columns={'LEAD TIME\n(DAYS)': 'Lead Time (Days)'})

final_df.head(10)

,PRODUCT,Dec-24,Jan-25,Feb-25,Mar-25,Apr-25,May-25,Jun-25,Jul-25,Aug-25,Sep-25,Oct-25,TotalSales_11Months,AdjustedAnnualSales,Lead Time (Days)
1,Ant Moore Chardy 150ml,65.0,87.0,86.0,102.0,56.0,36.0,34.0,75.0,42.0,47.0,73.0,703.0,767.0,3.0
2,Ant Moore Chardy 750ml,2.0,1.0,4.0,3.0,1.0,2.0,1.0,2.0,1.0,3.0,0.0,20.0,22.0,3.0
3,Ant Moore SB 150ml,33.0,69.0,59.0,84.0,54.0,20.0,23.0,49.0,49.0,33.0,44.0,517.0,564.0,3.0
4,Ant Moore Sauvignon Blanc 750ml,1.0,6.0,2.0,9.0,4.0,1.0,2.0,2.0,2.0,0.0,0.0,29.0,32.0,3.0
8,Brancott Brut 750ml,1.0,0.0,20.0,0.0,1.0,5.0,1.0,5.0,4.0,5.0,2.0,44.0,48.0,1.0
9,Brancott Cuvee 200ml,64.0,76.0,50.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,194.0,212.0,1.0
10,Brancott Estate Merlot 150ml,116.0,118.0,153.0,143.0,113.0,64.0,47.0,116.0,78.0,86.0,68.0,1102.0,1202.0,1.0
11,Brancott Estate Merlot 750ml,3.0,3.0,3.0,2.0,1.0,1.0,0.0,3.0,2.0,3.0,23.0,44.0,48.0,1.0
12,Bright Eyes White Pinot 150ml,27.0,42.0,34.0,39.0,23.0,17.0,3.0,12.0,13.0,29.0,17.0,256.0,279.0,1.0
13,Bright Eyes White Pinot 750ml,1.0,1.0,1.0,4.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,10.0,11.0,1.0


In [18]:
print(f"Rows before dropping wines with missing lead times: {final_df.shape[0]}")

#Drop rows where we are missing lead time information
final_df = final_df.dropna(subset=['Lead Time (Days)'])

print(f"Rows after dropping wines with missing lead times: {final_df.shape[0]}")

Rows before dropping wines with missing lead times: 148
Rows after dropping wines with missing lead times: 104


#### Next Step
- QC check on the matching to ensure we didnt match the incorrect row or that there aren't naming mismatches that are the same wine